In [3]:
#Reference: http://dlab.berkeley.edu/blog/scraping-new-york-times-articles-python-tutorial
#Reference: https://medium.freecodecamp.org/how-to-scrape-websites-with-python-and-beautifulsoup-5946935d93fe
from nytimesarticle import articleAPI
import csv #Import csv
#from bs4 import BeautifulSoup
#import re
import requests
import urllib.request
from bs4 import BeautifulSoup
import time
api = articleAPI('5569278a93bb41e89f68cb84499391a3')
start_index=1
end_index=3
j=start_index
# topics=['sports','entertainment','business','politics']
topics=['politics']
for search_word in topics :
    j=start_index
    while j < end_index:
        articles = api.search( q = search_word, page = j)
        #To get urls
#         urls = []
#         for i in articles['response']['docs']:
#             urls.append(i['web_url'])
#         for url in urls:
#             r = requests.get(url)
#             time.sleep(5)
#             soup = BeautifulSoup(r.content, 'html.parser')
#             for script in soup.find_all('script'):
#                 script.extract()
#             pTags = soup.findAll("p", text=True)
#             for paragraph in pTags:
#                 para=paragraph.get_text(strip=True)
#                 print(para)
#                 nytimesData.write(para)
        url_count = 0
        for i in articles['response']['docs']:
            filename=search_word+str(j)+str(url_count)+'.txt'
            nytimesData = open(filename, 'a+')
        
            url = i['web_url']
            r = requests.get(url)
            time.sleep(5)
            soup = BeautifulSoup(r.content, 'html.parser')
            for script in soup.find_all('script'):
                script.extract()
            pTags = soup.findAll("p", text=True)
            for paragraph in pTags:
                para=paragraph.get_text(strip=True)
                print(para)
                nytimesData.write(para)
            nytimesData.close()
            url_count += 1
        j += 1

ConnectionError: HTTPSConnectionPool(host='topics.nytimes.com', port=443): Max retries exceeded with url: /top/opinion/editorialsandoped/oped/contributors/thomasbedsall/index.html (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f7a908f6828>: Failed to establish a new connection: [Errno 111] Connection refused',))

ConnectionError: HTTPSConnectionPool(host='topics.nytimes.com', port=443): Max retries exceeded with url: /top/opinion/editorialsandoped/oped/contributors/thomasbedsall/index.html (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f7ab01fe588>: Failed to establish a new connection: [Errno 111] Connection refused',))